In [ ]:
import requests
import requests_cache
from requests_cache import CachedSession
from datetime import timedelta
import json
import time

In [ ]:
import osmnx as ox
import osm2geojson
from shapely.geometry import shape

In [ ]:
import ipywidgets as widgets

from IPython.display import clear_output
from ipyleaflet import SearchControl, FullScreenControl, MeasureControl, ScaleControl, LayersControl, WidgetControl, LegendControl
from ipyleaflet import Map, GeoJSON, MarkerCluster, Marker, AwesomeIcon, LayerGroup, link, basemaps

In [ ]:
query_dict = {}
query_dict['Toll Roads']= ' "toll"="yes" '
query_dict['EV Stations']= ' "amenity"="charging_station" '
query_dict['Gas Stations']= ' "amenity"="fuel" '
query_dict['Rail']= ' "railway"="rail" '
query_dict['Heliostat'] = ' "power":"heliostat" '
query_dict['Power Plant'] = ' "power":"plant" '
query_dict['Power Line'] = ' "power":"line" '
query_dict['Power Cable'] = ' "power":"cable" '
query_dict['Power Substation'] = ' "power":"substation" '
query_dict['Power Substation Minor'] = ' "substation":"minor_distribution" '
query_dict['Data Center'] = ' "telecom":"data_center" '
query_dict['Fuel Service'] = ' "highway":"service" '
query_dict['Nursing Home'] = ' "amenity":"nursing_home" '
query_dict['Recycling'] = ' "amenity":"recycling" '

In [ ]:
iconGas = AwesomeIcon(
    name='gas-pump',
    #marker_color='gray',
    #icon_color='white',
    spin=False
)

iconPlug = AwesomeIcon(
    name='plug',
    marker_color='green',
    icon_color='white',
    spin=False
)

iconCharging = AwesomeIcon(
    name='charging-station',
    #marker_color='green',
    #icon_color='white',
    spin=False
)

iconSolar = AwesomeIcon(
    name='solar-panel',
    marker_color='green',
    icon_color='white',
    spin=False
)

iconPowerPlant = AwesomeIcon(
    name='industry',
    marker_color='gray',
    icon_color='white',
    spin=False
)


In [ ]:
def query_overpass_roads(session, boundary, search_terms):
    """Timeout set to 300 seconds (5 mins). Default timeout is 180.
    Max size is 1 GB: [maxsize:1073741824]"""

    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:300];
    (
      node["highway"][{search_terms}]{boundary};
      way["highway"][{search_terms}]{boundary};
      relation["highway"][{search_terms}]{boundary};
    );
    out body;
    >;
    out skel qt;
    """
    
    try:
        time.sleep( 3 )
        response = session.get(overpass_url, params = {'data': overpass_query} )

        #response = requests.get(overpass_url, 
        #                    params={'data': overpass_query})

        response.raise_for_status()
        # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)

    return response

In [ ]:
def query_overpass(session, boundary, search_terms):
    """Timeout set to 300 seconds (5 mins). Default timeout is 180.
    Max size is 1 GB: [maxsize:1073741824]"""

    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:300];
    (
      node[{search_terms}]{boundary};
      way[{search_terms}]{boundary};
      relation[{search_terms}]{boundary};
    );
    out body;
    >;
    out skel qt;
    """
    
    try:
        time.sleep( 3 )
        response = session.get(overpass_url, params = {'data': overpass_query} )

        #response = requests.get(overpass_url, 
        #                    params={'data': overpass_query})

        response.raise_for_status()
        # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)

    return response

In [ ]:
def get_geojson_widget(geojson, name, color='darkblue'):
    
    for feature in geojson['features']:
        feature['properties']['style'] = {
            'color': color,
            'weight': 1,
            'fillColor': 'lightblue',
            'fillOpacity': 0.5
        }
    
    leaflet_geojson = GeoJSON(
        name = name,
        data=geojson,
        # below is tiles style
        style={ 'color':color, 'opacity': 1, 'weight': 1, 'dashArray': '9','fillOpacity': 0.1},
    )
    
    return leaflet_geojson

In [ ]:
def add_marker_cluster(geojson3, icon=None, name='Marker Cluster'):
    markers = []
    for i in range(len(geojson3['features'])):
        pp = shape( geojson3['features'][i]['geometry']).centroid
        location = ([pp.y, pp.x])
        marker = Marker(location=location, icon=icon)
        markers.append(marker)

    cluster = MarkerCluster(name = name, markers = markers, max_cluster_radius=100) 
    # disable_clustering_at_zoom=3, 
    # default radius 80 pixels, Decreasing will make more, smaller clusters.
    return cluster

In [ ]:
def get_map_boundary(m):
    (i1, i2), (j1, j2) = m.bounds
    boundary = (i1, i2, j1, j2)
    return boundary

In [ ]:
align_kw = dict(
    _css = (('.widget-label', 'min-width', '20ex'),),
    margin = '0px 0px 5px 12px'
)

button_toll = widgets.Button(description="Query Toll Roads", **align_kw)
button_rail = widgets.Button(description="Query Rail Roads", **align_kw)
button_ev = widgets.Button(description="Query EV Stations", **align_kw)
button_gas = widgets.Button(description="Query Gas Stations", **align_kw)

In [ ]:
session = CachedSession(allowable_codes=[200], old_data_on_error=True, backend='sqlite', expire_after=timedelta(days=30))
session.mount('http://overpass-api.de', requests.adapters.HTTPAdapter(max_retries = 2))

In [ ]:
m = Map(center=(43.651070, -79.347015), zoom=10) #, basemap=basemaps.Esri.WorldStreetMap

In [ ]:
#m.basemap = basemaps.Stamen.Terrain
#m.basemap = basemaps.Esri.WorldStreetMap
#m.basemap = basemaps.Esri.DeLorme
#Important Note: This item is in mature support as of September 2020 and is no longer updated. 

In [ ]:
search = SearchControl(position="topleft", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=10,
                       property_name='display_name',
                       marker=None
                      )
m.add_control(search)

In [ ]:
fullscreen = FullScreenControl()
m.add_control(fullscreen)

In [ ]:
scale = ScaleControl(position = 'bottomleft', imperial=False)
m.add_control(scale)

In [ ]:
measure = MeasureControl(position='bottomleft', active_color='orange', primary_length_unit = 'kilometers')
m.add_control(measure)

In [ ]:
minimap = Map(
    zoom_control=False, attribution_control=False, 
    zoom=3, center=m.center, basemap=basemaps.Stamen.Terrain
)
minimap.layout.width = '150px'
minimap.layout.height = '150px'
link((minimap, 'center'), (m, 'center'))
minimap_control = WidgetControl(widget=minimap, position='bottomright')
m.add_control(minimap_control)

In [ ]:
def get_toll_roads(b):
    global session
    global m
    case = 'Toll Roads'
    boundary = get_map_boundary(m)
    response = query_overpass_roads(session, boundary, query_dict[case] )
    geojson = osm2geojson.json2geojson( response.json() )
    toll_roads = get_geojson_widget(geojson, case, color="navy")
    m.add_layer( toll_roads )

In [ ]:
button_toll.on_click( get_toll_roads )


In [ ]:
def get_ev_charg_stations(b):
    global session
    global m
    case = 'EV Stations'
    boundary = get_map_boundary(m)
    response2 = query_overpass(session, boundary, query_dict[case] )
    geojson2 = osm2geojson.json2geojson( response2.json() )
    ev_charg_layer = add_marker_cluster(geojson2, icon=iconCharging, name=case)
    m.add_layer(ev_charg_layer)

In [ ]:
button_ev.on_click( get_ev_charg_stations )


In [ ]:
def get_gas_stations(b):
    global session
    global m
    case = 'Gas Stations'
    boundary = get_map_boundary(m)
    response3 = query_overpass(session, boundary, query_dict[case] )
    geojson3 = osm2geojson.json2geojson( response3.json() )
    fuel_layer = add_marker_cluster(geojson3, icon=iconGas, name=case)
    m.add_layer(fuel_layer)

In [ ]:
button_gas.on_click( get_gas_stations )


In [ ]:
def get_rail_roads(b):
    global session
    global m
    case = 'Rail'
    boundary = get_map_boundary(m)
    response4 = query_overpass(session, boundary, query_dict[case] )
    geojson4 = osm2geojson.json2geojson( response4.json() )
    rail = get_geojson_widget(geojson4, case, color="steelblue")
    m.add_layer( rail )

In [ ]:
button_rail.on_click( get_rail_roads )


In [ ]:
instructions = widgets.HTML(
    value="""
    <p>Search a city and plot infrastructure assets: a query for one city takes 3-10 seconds.<br> 
    <hr>
    """
)


In [ ]:
footnote = widgets.HTML(value=
                       """
                        <h1>Big Spatial Data Analytics</h1>
                        
                        <p>This web app enables easy access to OpenStreetMap, which contains over 5 billion locations worldwide, incl. 
                        <b>transport</b> infrastructure (roads, fuel stations),
                        <b>power</b> infrastructure (power plants, power lines),
                        <b>data</b> infrastructure (data centers),
                        and infrastructure <b>plans</b> (landuse classification)</p>
                        
                        <p>A query for a large city takes 10-30 seconds</p>
                        """
                      )

# Page

## Section

### Look up a city and discover infrastructure assets

In [ ]:
widgets.VBox([m])

# Sidebar

### 

In [ ]:
footnote

In [ ]:
widget_control_1 = WidgetControl(widget=button_toll, position='topright')
m.add_control(widget_control_1)

widget_control_2 = WidgetControl(widget=button_gas, position='topright')
m.add_control(widget_control_2)

widget_control_3 = WidgetControl(widget=button_ev, position='topright')
m.add_control(widget_control_3)

widget_control_4 = WidgetControl(widget=button_rail, position='topright')
m.add_control(widget_control_4)

In [ ]:
legend = LegendControl({"Toll Roads":"navy", "Rail Roads":"steelblue"}, position="bottomright")
m.add_control(legend)

In [ ]:
control = LayersControl(position='topright', collapsed=False)
m.add_control(control)